### StructuredOuptutParser
* LLM에서 나오는 답변을 dict 형식으로 구성을 해준다.
* key/value 쌍으로 필드를 반환하고 싶을 때 사용할 수 있다.
* GPT-4 또는 Calude 모델보다 파라미터수가 적은 로컬모델들 같은 경우에 효과적이다.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schema = [
    ResponseSchema(name="answer", description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 여야한다."
    )
]

# 응답 스키마를 기반으로 한 StructuredOutputParser 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [4]:
from langchain_core.prompts import PromptTemplate

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

prompt = PromptTemplate(
    template="answer the question as beset as possible\nFormat:\n{format_instructions}\nQuestion:\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 여야한다.
}
```


In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser

In [11]:
chain.invoke({"question": "원신에서 만개를 활용한 파티를 사용하고 싶은데, 어떤 캐릭터들로 구성하는게 좋을까?"})

{'answer': '만개를 활용한 파티를 구성할 때는 주로 물 속성 캐릭터들을 활용하는 것이 좋습니다. 만개는 물 속성 원소 구슬을 생성하고 물 원소 피해를 증가시키는 능력을 가지고 있기 때문입니다. 만개와 함께 사용하기 좋은 캐릭터로는 행추, 모나, 자인, 바바라 등이 있습니다.',
 'source': 'https://www.thisisagoodsource.com'}